<a href="https://colab.research.google.com/github/swaminaathakrishnan/Cool_Route_prototype/blob/master/Cool_route_v5.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚴 **CoolRide V5.2: User Input Edition**
### Project Overview

CoolRide V5.2 adds **user input** for custom routes!

**New in V5.2:**
- 📍 User input for starting point (latitude, longitude)
- 🎯 User input for ending point (latitude, longitude)
- ⏰ Optional time input (defaults to current time if not provided)
- 🎨 Interactive route customization

**Features from V5:**
- 💧 Water body cooling effects (evaporative + convective cooling)
- 🌊 100m buffer zone for water-adjacent routes
- 📏 Size-based filtering (excludes tiny ponds <0.1 km²)
- 🎯 45% cost reduction for water-adjacent paths
- 🏢 Building shadow calculation using sun position
- ⏰ Time-aware routing (different routes at different times of day)
- 🌡️ Combined tree + building shade analysis

CoolRide is a TRL-6 prototype designed to route cyclists through thermally comfortable paths in Singapore. It utilizes real-time government weather data (NEA), building shadows, vegetation data, water bodies, and predictive AI to mitigate Urban Heat Island (UHI) effects.

In [1]:
# ==========================================
# 🧱 MODULE 1: SYSTEM INITIALIZATION
# ==========================================
# Objective: Install geospatial libraries and set up the environment.
# dependencies: OSMnx (Maps), GeoPandas (Spatial Data), Scikit-Learn (AI).

!pip install osmnx simplekml geopandas shapely networkx requests scikit-learn -q

import osmnx as ox
import networkx as nx
import simplekml
import geopandas as gpd
import pandas as pd
import requests
import numpy as np
import os
import pickle
import time
from shapely.geometry import Point, LineString, box
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

print("✅ System Initialized. Ready for V5 Execution.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 4.3 MB/s eta 0:00:00
✅ System Initialized. Ready for V5 Execution.


### ⚙️ Module 2: Configuration & Cloud Connection
Objective: Define the pilot zone and connect to the GitHub Data Lake. Logic: Instead of local files, we stream GeoJSON/CSV directly from the raw GitHub URLs. This allows the team to collaborate without sharing Drive folders.

In [9]:
# ==========================================
# ⚙️ MODULE 2: CONFIGURATION (V5.3 - AUTOMATION READY)
# ==========================================
import pytz
from datetime import datetime

print("🚴 COOLRIDE V5.3 - PRODUCTION ENGINE")
print("=" * 50)

# 1. ROUTE PARAMETERS (Bridge Variables)
# Ziyi's App can overwrite these lines automatically before execution.
# Default: Tampines Loop
START_NAME = "Tampines MRT"
START_COORDS = (1.3533, 103.9452)

END_NAME = "Tampines Eco Green"
END_COORDS = (1.3598, 103.9351)

# 2. TIME CONFIGURATION (CRITICAL FIX: UTC+8)
# We force the timezone to Asia/Singapore so shadows are accurate.
sgt_zone = pytz.timezone('Asia/Singapore')
current_time_sgt = datetime.now(sgt_zone)

# Manual Override for Demo (Optional)
# DEPARTURE_TIME = current_time_sgt.replace(hour=14, minute=0)

# Force 2:00 PM today for shadow simulation
DEPARTURE_TIME = current_time_sgt.replace(hour=14, minute=0, second=0)
# DEPARTURE_TIME = current_time_sgt

PLACE_NAME = f"{START_NAME} to {END_NAME}"
START_POINT = START_COORDS
END_POINT = END_COORDS

print(f"   📍 Route: {PLACE_NAME}")
print(f"   🕐 Departure Time (SGT): {DEPARTURE_TIME.strftime('%I:%M %p')}")
print(f"      (Sun position calculated for: {DEPARTURE_TIME})")

# 3. GITHUB DATA LAKE
GITHUB_USER = "swaminaathakrishnan"
REPO_NAME = "Cool_Route_prototype"
BASE_URL = f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/master/data/"

# File Links
PCN_URL = BASE_URL + "ParkConnectorLoop.geojson"
HAWKER_URL = BASE_URL + "HawkerCentresGEOJSON.geojson"
TREES_URL = BASE_URL + "trees.csv"
WATER_URL = BASE_URL + "water.geojson"

# 4. THERMAL WEIGHTS
WEIGHT_PCN = 0.5
WEIGHT_WATER = 0.55
WEIGHT_TREE_SHADE = 0.6
WEIGHT_BUILDING_SHADE = 0.7
WEIGHT_ULTIMATE = 0.35 # All factors combined

# 5. WATER PARAMETERS
WATER_BUFFER_DISTANCE = 100
MIN_WATER_SIZE = 50000

# 6. SAFETY OVERRIDE
NEA_HEATWAVE_ALERT = False

print("=" * 50)

🚴 COOLRIDE V5.3 - PRODUCTION ENGINE
   📍 Route: Tampines MRT to Tampines Eco Green
   🕐 Departure Time (SGT): 02:00 PM
      (Sun position calculated for: 2025-12-23 14:00:00.132865+08:00)


### Module 3A: Sun Position & Building Shadow Engine


Objective: Calculate real-time building shadows based on sun position and building heights.

**Physics:**
- Sun elevation angle determines shadow length
- Shadow direction is opposite of sun azimuth
- Taller buildings cast longer shadows (especially at low sun angles)

In [5]:

# MODULE 3A: SUN POSITION & BUILDING SHADOWS (V4)

def calculate_sun_position(latitude, longitude, timestamp):
    """
    Calculate sun elevation and azimuth for given location and time

    Returns: (elevation_degrees, azimuth_degrees)
    """
    import math
    day_of_year = timestamp.timetuple().tm_yday

    # Declination angle
    declination = 23.45 * math.sin(math.radians((360/365) * (day_of_year - 81)))

    # Hour angle
    hour = timestamp.hour + timestamp.minute / 60.0
    hour_angle = 15 * (hour - 12)

    # Sun elevation
    lat_rad = math.radians(latitude)
    dec_rad = math.radians(declination)
    ha_rad = math.radians(hour_angle)

    sin_elev = (math.sin(lat_rad) * math.sin(dec_rad) +
                math.cos(lat_rad) * math.cos(dec_rad) * math.cos(ha_rad))
    elevation = math.degrees(math.asin(max(-1, min(1, sin_elev))))

    # Sun azimuth
    cos_azim = ((math.sin(dec_rad) - math.sin(lat_rad) * sin_elev) /
                (math.cos(lat_rad) * math.cos(math.radians(elevation))))
    cos_azim = max(-1, min(1, cos_azim))
    azimuth = math.degrees(math.acos(cos_azim))

    if hour > 12:
        azimuth = 360 - azimuth

    return elevation, azimuth


def create_shadow_polygon(building_polygon, building_height, sun_elevation, sun_azimuth):
    """
    Create shadow polygon from building footprint

    Args:
        building_polygon: Shapely Polygon
        building_height: Height in meters
        sun_elevation: Sun angle above horizon (degrees)
        sun_azimuth: Sun compass direction (degrees)

    Returns:
        Shadow polygon (Shapely)
    """
    import math
    from shapely.affinity import translate

    if sun_elevation <= 0:
        return None  # Night time

    # Shadow length = height / tan(elevation)
    shadow_length = building_height / math.tan(math.radians(sun_elevation))

    # Shadow direction (opposite of sun)
    shadow_direction = (sun_azimuth + 180) % 360

    # Calculate offset in meters
    shadow_offset_y = shadow_length * math.cos(math.radians(shadow_direction))
    shadow_offset_x = shadow_length * math.sin(math.radians(shadow_direction))

    # Get building centroid
    centroid = building_polygon.centroid
    lat, lon = centroid.y, centroid.x

    # Convert meters to degrees
    deg_per_meter_lat = 1 / 111000
    deg_per_meter_lon = 1 / (111000 * math.cos(math.radians(lat)))

    offset_lat = shadow_offset_y * deg_per_meter_lat
    offset_lon = shadow_offset_x * deg_per_meter_lon

    # Create shadow by translating building polygon
    shadow = translate(building_polygon, xoff=offset_lon, yoff=offset_lat)

    # Union with building for full coverage
    full_shadow = building_polygon.union(shadow).convex_hull

    return full_shadow

print("Sun Position & Building Shadow Functions Loaded")

Sun Position & Building Shadow Functions Loaded


### Module 3B: Enhanced Spatial Graph Engine

Objective: Build road network and overlay ALL cooling features.

**Layer 1:** Road Network (OSM)  
**Layer 2:** Park Connectors (PCN)  
**Layer 3:** Tree Canopy (SGTrees)  
**Layer 4:** Building Shadows (Time-dependent)  
**Layer 5:** Water Bodies (NEW in V5! - Proximity-based cooling)

Logic: Roads receive cumulative discounts based on shade coverage from multiple sources.

In [13]:
# ==========================================
# 🗺️ MODULE 3B: ENHANCED SPATIAL GRAPH ENGINE (V5.4 - ROBUST WATER)
# ==========================================

def generate_cool_routes():
    print(f"⏳ Downloading road network for {PLACE_NAME}...")

    # 1. GET GRAPH
    try:
        G = ox.graph_from_point(START_POINT, dist=2000, network_type='bike')
        nodes = ox.graph_to_gdfs(G, edges=False)
        miny, maxy = nodes.y.min(), nodes.y.max()
        minx, maxx = nodes.x.min(), nodes.x.max()
        print(f"   📐 Zone Limits: Lat[{miny:.4f}, {maxy:.4f}], Lon[{minx:.4f}, {maxx:.4f}]")
    except Exception as e:
        print(f"   ❌ Network Error: {e}")
        return None, None, None, None

    # 2. LOAD PCN
    print("⏳ Overlaying Park Connectors...")
    pcn_union = None
    try:
        pcn_data = gpd.read_file(PCN_URL)
        if pcn_data.crs != "EPSG:4326": pcn_data = pcn_data.to_crs("EPSG:4326")
        try: pcn_union = pcn_data.geometry.union_all()
        except: pcn_union = pcn_data.geometry.unary_union
        print(f"   ✅ PCN Loaded")
    except:
        print("   ⚠️ PCN Data missing (Proceeding without it)")

    # 3. LOAD TREES
    print("⏳ Loading Tree Canopy Data...")
    trees_buffer = None
    try:
        local_file = "trees_downloaded.csv"
        if not os.path.exists(local_file):
            lfs_url = f"https://github.com/{GITHUB_USER}/{REPO_NAME}/raw/master/data/trees.csv"
            os.system(f"wget -O {local_file} {lfs_url}")

        trees_df = pd.read_csv(local_file)
        trees_df.columns = [c.lower() for c in trees_df.columns]
        lat_col = 'latitude' if 'latitude' in trees_df.columns else 'lat'
        lng_col = 'longitude' if 'longitude' in trees_df.columns else 'lng'

        trees_df = trees_df[
            (trees_df[lat_col] >= miny) & (trees_df[lat_col] <= maxy) &
            (trees_df[lng_col] >= minx) & (trees_df[lng_col] <= maxx)
        ]

        if len(trees_df) > 0:
            geometry = [Point(xy) for xy in zip(trees_df[lng_col], trees_df[lat_col])]
            trees_gdf = gpd.GeoDataFrame(trees_df, geometry=geometry, crs="EPSG:4326")
            trees_buffer = trees_gdf.geometry.buffer(0.0001).unary_union
            print(f"   ✅ Tree shade layer generated ({len(trees_df)} trees)")
    except Exception as e:
        print(f"   ⚠️ Tree Data Error: {e}")

    # 4. LOAD BUILDINGS (With SGT Time)
    print("⏳ Loading Buildings...")
    building_shadows = None
    try:
        buildings_gdf = ox.features_from_point(START_POINT, tags={'building': True}, dist=2000)
        buildings_gdf = buildings_gdf[buildings_gdf.geometry.type == 'Polygon']

        # Height Logic
        buildings_gdf['estimated_height'] = 15

        # Calculate Sun Position
        sun_elev, sun_azim = calculate_sun_position(START_POINT[0], START_POINT[1], DEPARTURE_TIME)

        print(f"   ☀️ Sun (SGT): {sun_elev:.1f}° elev, {sun_azim:.1f}° azim")

        if sun_elev > 0:
            shadow_polygons = []
            for _, building in buildings_gdf.iterrows():
                shadow = create_shadow_polygon(building.geometry, building['estimated_height'], sun_elev, sun_azim)
                if shadow: shadow_polygons.append(shadow)

            from shapely.ops import unary_union
            building_shadows = unary_union(shadow_polygons)
            print(f"   ✅ Building shadow layer generated")
        else:
            print("   🌙 Night time (No shadows)")

    except Exception as e:
        print(f"   ⚠️ Building Error: {e}")

    # 5. LOAD WATER BODIES (SPLIT-FETCH STRATEGY)
    print("⏳ Loading Water Bodies...")
    water_buffer = None
    water_gdfs = []

    # A. Try GitHub Download First
    try:
        local_water = "data/water.geojson"
        if not os.path.exists(local_water):
             resp = requests.get(WATER_URL, timeout=3)
             if resp.status_code == 200:
                 with open(local_water, 'wb') as f: f.write(resp.content)
                 gdf = gpd.read_file(local_water)
                 water_gdfs.append(gdf)
                 print("   📥 Loaded Water from GitHub")
    except: pass

    # B. Fallback to OSM (Split by Tag to prevent total failure)
    if not water_gdfs:
        print("   🌍 Fetching Water from OpenStreetMap (Split Strategy)...")
        # We fetch layers separately. If 'canals' crash, we still keep 'lakes'.
        tags_to_try = [
            {'natural': 'water'},       # Lakes/Reservoirs (Safe)
            {'waterway': 'river'},      # Rivers (Medium)
            {'waterway': 'canal'}       # Canals (High Risk of NaN errors)
        ]

        bbox = (maxy, miny, maxx, minx)

        for tag in tags_to_try:
            try:
                gdf = ox.features_from_bbox(bbox=bbox, tags=tag)
                if not gdf.empty:
                    # Strict Filter: Polygons/Lines only (Remove Points which cause errors)
                    gdf = gdf[gdf.geometry.type.isin(['Polygon', 'MultiPolygon', 'LineString', 'MultiLineString'])]
                    if not gdf.empty:
                        water_gdfs.append(gdf)
            except:
                continue # If one tag fails, skip it and keep going

    # C. Combine & Buffer
    if water_gdfs:
        try:
            # Merge all successful downloads
            water_gdf = pd.concat(water_gdfs, ignore_index=True)
            if not water_gdf.empty:
                if water_gdf.crs != "EPSG:4326": water_gdf = water_gdf.to_crs("EPSG:4326")

                # Buffer ~100m
                water_buffer = water_gdf.geometry.buffer(0.001).unary_union
                print(f"   ✅ Water Cooling Layer Generated ({len(water_gdf)} features)")
            else:
                print("   ℹ️ No significant water bodies found (Filtered)")
        except Exception as e:
            print(f"   ⚠️ Water Merge Error: {e}")
    else:
        print("   ℹ️ No water bodies found")

    # 6. CALCULATE COST
    print("⏳ Calculating Costs...")

    # Time-dependent building shade factor
    hour = DEPARTURE_TIME.hour
    shade_multiplier = 0.6 if (hour < 10 or hour > 16) else 1.0

    for u, v, k, data in G.edges(keys=True, data=True):
        if 'geometry' in data: edge_geom = data['geometry']
        else: edge_geom = LineString([(G.nodes[u]['x'], G.nodes[u]['y']), (G.nodes[v]['x'], G.nodes[v]['y'])])

        cost = data['length']
        tags = []

        # Factors
        is_pcn = pcn_union and edge_geom.intersects(pcn_union)
        is_tree = trees_buffer and edge_geom.intersects(trees_buffer)
        is_shadow = building_shadows and edge_geom.intersects(building_shadows)
        is_water = water_buffer and edge_geom.intersects(water_buffer)

        # Scoring Logic
        if is_tree and is_shadow and is_water:
            cost *= WEIGHT_ULTIMATE
            tags.append("🌳🏢💧 Ultimate Cool")
        elif is_tree and is_shadow:
            cost *= 0.45
            tags.append("🌳🏢 Tree+Bldg")
        elif is_water:
            cost *= WEIGHT_WATER
            tags.append("💧 Water Breeze")
        elif is_tree:
            cost *= WEIGHT_TREE_SHADE
            tags.append("🌳 Tree Shade")
        elif is_shadow:
            cost *= (WEIGHT_BUILDING_SHADE * shade_multiplier)
            tags.append("🏢 Bldg Shadow")
        elif is_pcn:
            cost *= WEIGHT_PCN
            tags.append("🌿 PCN")

        data['cool_cost'] = cost
        data['tag'] = " + ".join(tags) if tags else "☀️ Exposed"

    # 7. SOLVE
    orig = ox.distance.nearest_nodes(G, START_POINT[1], START_POINT[0])
    dest = ox.distance.nearest_nodes(G, END_POINT[1], END_POINT[0])

    try:
        r_fast = nx.shortest_path(G, orig, dest, weight='length')
        r_cool = nx.shortest_path(G, orig, dest, weight='cool_cost')
        return G, r_fast, r_cool, []
    except Exception as e:
        print(f"   ❌ Routing failed: {e}")
        return None, None, None, None

In [11]:
# ==========================================
# 🧠 MODULE 4: HISTORICAL AI ENGINE (V3)
# ==========================================
CACHE_FILE = "coolride_weather_memory.pkl"

def get_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, "rb") as f: return pickle.load(f)
        except: return {}
    return {}

def save_cache(data):
    with open(CACHE_FILE, "wb") as f: pickle.dump(data, f)

def fetch_historical_data(station_name, days_back=3):
    cache = get_cache()
    today_str = datetime.now().strftime("%Y-%m-%d")
    cache_key = f"{station_name}_{today_str}"

    if cache_key in cache and len(cache[cache_key]['values']) > 20:
        print(f"   ⚡ Memory Hit! Loaded {len(cache[cache_key]['values'])} points.")
        return cache[cache_key]['timestamps'], cache[cache_key]['values']

    print(f"   📡 Memory Miss. Analyzing last {days_back} days...")
    all_timestamps, all_values = [], []

    for i in range(days_back + 1):
        target_date = (datetime.now() - timedelta(days=i)).strftime("%Y-%m-%d")
        url = "https://api-open.data.gov.sg/v2/real-time/api/weather"
        params = {"api": "wbgt", "date": target_date}

        # PAGINATION LOOP
        while True:
            try:
                resp = requests.get(url, params=params, timeout=5)
                if resp.status_code != 200: break
                data = resp.json()
                if 'data' not in data: break

                for rec in data['data'].get('records', []):
                    dt = datetime.fromisoformat(rec['datetime'])
                    for r in rec['item']['readings']:
                        if r.get('station', {}).get('name') == station_name:
                            val = float(r.get('wbgt') or r.get('value'))
                            mins = dt.hour * 60 + dt.minute
                            now_mins = datetime.now().hour * 60 + datetime.now().minute
                            if abs(mins - now_mins) < 240: # 4 hour window
                                all_timestamps.append(mins)
                                all_values.append(val)

                token = data['data'].get('paginationToken')
                if token: params['paginationToken'] = token
                else: break
            except: break

    if len(all_values) > 20:
        cache[cache_key] = {'timestamps': all_timestamps, 'values': all_values}
        save_cache(cache)
        print(f"   💾 Learned & Saved {len(all_values)} thermal patterns.")

    return all_timestamps, all_values

def predict_trend(station_name, current_wbgt):
    timestamps, values = fetch_historical_data(station_name)
    if len(values) < 10: return current_wbgt, "Stable ➖", "Low Data"

    # Linear Regression
    model = LinearRegression()
    model.fit(np.array(timestamps).reshape(-1, 1), np.array(values))

    # Forecast
    now = datetime.now()
    fut_min = now.hour * 60 + now.minute + 15
    raw_pred = model.predict([[fut_min]])[0]

    # Physics Clamp
    delta = raw_pred - current_wbgt
    if abs(delta) > 0.5:
        final_pred = current_wbgt + (0.5 if delta > 0 else -0.5)
        note = "(Physics Clamped)"
    else:
        final_pred = raw_pred
        note = ""

    trend = "Rising 📈" if final_pred > current_wbgt + 0.1 else "Falling 📉" if final_pred < current_wbgt - 0.1 else "Stable ➖"
    return final_pred, trend, f"High {note}"

### 🚀 Module 5: Execution & Safe-Pace Recommendations

Objective: Synthesize map, weather, and AI data into a KML route. Upgrade:

* Govt Override: Checks NEA_HEATWAVE_ALERT.
* Safe Pacing: Suggests specific ride speeds and hydration intervals based on WBGT (ISO 7243 standards).

In [14]:
# ==========================================
# 🚀 MODULE 5: EXECUTION ENGINE (V3.9 - FINAL LABELS)
# ==========================================
import math
import os
import time
import requests
import simplekml

print("🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...")

# --- HELPER: DYNAMIC SENSOR FINDER (FULL V2 LOGIC) ---
def get_nearest_wbgt_station(lat, lon):
    print("⏳ Connecting to NEA Official WBGT Sensor Network...")
    url = "https://api-open.data.gov.sg/v2/real-time/api/weather"
    try:
        resp = requests.get(url, params={"api": "wbgt"}, timeout=10)
        data = resp.json()
        readings = data['data']['records'][0]['item'].get('readings', [])

        closest_station = "Unknown"
        min_dist = float('inf')
        current_val = None

        for r in readings:
            try:
                loc = {}
                s_name = "Unknown"
                if 'location' in r: loc = r['location']
                elif 'station' in r and 'location' in r['station']: loc = r['station']['location']
                if 'station' in r: s_name = r['station'].get('name', 'Unknown')
                s_lat = float(loc.get('latitude', 0))
                s_lon = float(loc.get('longitude', loc.get('longtitude', 0)))
                if s_lat == 0 or s_lon == 0: continue

                val = r.get('wbgt') or r.get('value')
                if val is None: continue
                val = float(val)
                dist = math.sqrt((lat - s_lat)**2 + (lon - s_lon)**2)

                if dist < min_dist:
                    min_dist = dist
                    closest_station = s_name
                    current_val = val
            except: continue

        if current_val is None: return 30.0, "System Fallback"
        print(f"   📍 Nearest Sensor: {closest_station} (Dist: {min_dist*111:.2f} km)")
        return current_val, closest_station
    except Exception as e:
        print(f"   ⚠️ WBGT Sensor Fail: {e}. Using Default Safety Value.")
        return 30.0, "System Fallback"

# 1. GENERATE ROUTES
graph, r1, r2, shelters = generate_cool_routes()

if graph:
    # 2. GET WEATHER
    current_wbgt, station_name = get_nearest_wbgt_station(START_POINT[0], START_POINT[1])

    # 3. RUN AI
    pred_wbgt, trend, confidence = predict_trend(station_name, current_wbgt)
    effective_wbgt = max(current_wbgt, pred_wbgt)
    if NEA_HEATWAVE_ALERT: effective_wbgt = 35.0

    # 4. REPORT
    if effective_wbgt < 29: rec = "✅ Safe to Ride."
    elif effective_wbgt < 31: rec = "⚠️ CAUTION: Seek shade."
    else: rec = "🛑 HIGH RISK: Stop."

    print(f"\n📊 REPORT: {station_name}")
    print(f"   Current: {current_wbgt}°C | Forecast: {pred_wbgt:.1f}°C")

    # 5. EXPORT KML (CLEAN LABELS)
    kml = simplekml.Kml()

    def add_route(route, color, name, description):
        ls = kml.newlinestring(name=name)
        coords = []
        for u, v in zip(route[:-1], route[1:]):
            d = graph.get_edge_data(u, v)[0]
            if 'geometry' in d:
                xs, ys = d['geometry'].xy
                coords.extend(list(zip(xs, ys)))
            else:
                coords.append((graph.nodes[u]['x'], graph.nodes[u]['y']))
                coords.append((graph.nodes[v]['x'], graph.nodes[v]['y']))
        ls.coords = coords
        ls.style.linestyle.color = color
        ls.style.linestyle.width = 5
        ls.description = description

    # 🧠 FUZZY LOGIC + CLEAN LABELS
    def check_similarity(route_a, route_b):
        set_a = set(route_a)
        set_b = set(route_b)
        intersection = len(set_a.intersection(set_b))
        union = len(set_a.union(set_b))
        return intersection / union

    sim_score = check_similarity(r1, r2)
    print(f"   🔍 Route Similarity Score: {sim_score*100:.1f}%")

    if sim_score > 0.90:  # If >90% similar, merge them
        print("   💡 Insight: Routes are effectively identical (Merged).")
        # MERGED LABEL
        add_route(r2, simplekml.Color.green, "🌟 Recommended Route",
                  f"<b>Smart Choice</b><br>The fastest path is also the coolest.<br>No detour needed.<br><br>Temp: {effective_wbgt:.1f}°C")
    else:
        print("   💡 Insight: A distinct cooler detour exists.")
        # DIVERGENT LABELS
        add_route(r1, simplekml.Color.red, "⚡ Fastest Route (Exposed)",
                  f"<b>Direct Path</b><br>Shortest time, but higher heat exposure.<br><br>Temp: {effective_wbgt:.1f}°C")
        add_route(r2, simplekml.Color.green, "🌿 Cool Route (Shaded)",
                  f"<b>Shaded Detour</b><br>Maximized tree canopy coverage.<br>Lower heat stress.<br><br>Temp: {effective_wbgt:.1f}°C")

    # Add Shelters
    if shelters:
        for name, lat, lon in shelters:
            p = kml.newpoint(name=f"🧊 {name}", coords=[(lon, lat)])
            p.style.iconstyle.icon.href = 'http://googleusercontent.com/maps.google.com/mapfiles/kml/shapes/snowflake_simple.png'

    # 6. SAVE
    if not os.path.exists('output'): os.makedirs('output')
    constant_filename = "output/latest_route.kml"
    kml.save(constant_filename)

    print(f"\n🎉 SUCCESS! Download '{constant_filename}'")
else:
    print("❌ Critical Error: Route Generation Failed.")

🚀 STARTING COOLRIDE ENGINE (WITH ACTIVE AI)...
⏳ Downloading road network for Tampines MRT to Tampines Eco Green...
   📐 Zone Limits: Lat[1.3353, 1.3713], Lon[103.9273, 103.9632]
⏳ Overlaying Park Connectors...
   ✅ PCN Loaded
⏳ Loading Tree Canopy Data...


/tmp/ipython-input-2525741080.py:53: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  trees_buffer = trees_gdf.geometry.buffer(0.0001).unary_union
/tmp/ipython-input-2525741080.py:53: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  trees_buffer = trees_gdf.geometry.buffer(0.0001).unary_union


   ✅ Tree shade layer generated (29766 trees)
⏳ Loading Buildings...
   ☀️ Sun (SGT): 51.7° elev, 227.8° azim
   ✅ Building shadow layer generated
⏳ Loading Water Bodies...
   🌍 Fetching Water from OpenStreetMap (Split Strategy)...
   ℹ️ No water bodies found
⏳ Calculating Costs...


/usr/local/lib/python3.12/dist-packages/shapely/measurement.py:50: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)
/usr/local/lib/python3.12/dist-packages/shapely/measurement.py:50: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)
/usr/local/lib/python3.12/dist-packages/shapely/measurement.py:50: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)


⏳ Connecting to NEA Official WBGT Sensor Network...
   📍 Nearest Sensor: Bedok North Street 2 (Dist: 3.12 km)
   ⚡ Memory Hit! Loaded 128 points.

📊 REPORT: Bedok North Street 2
   Current: 25.5°C | Forecast: 26.0°C
   🔍 Route Similarity Score: 93.1%
   💡 Insight: Routes are effectively identical (Merged).

🎉 SUCCESS! Download 'output/latest_route.kml'
